__Andrew Kittredge <br>
GTECH 731 <br>
Assignment 3__

__Goal:__ Improve and restructure code from assignment 2
- Discover, design, create reusable functions
- Improve brevity and efficiency using comprehensions for lists, dictionaries, sets, and for generator expressions
- Add necessary comments and documentation to clarify logic and key steps

__Task 1:__ Load data files. Add state names and abbreviations before further analysis.

In [1]:
import requests, json

# Load JSON files
with open("stateCodeToFips.json", 'r') as a, open("fipsToState.json", 'r') as b:
    codeToFips = json.load(a)
    fipsToState = json.load(b)

url = "https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_5m.json"

with requests.get(url) as response:
    data = response.json()  
    
# Invert code to fips dictionary using dictionary comprehension
fipsToCode = {v: k for k, v in codeToFips.items()}

# Extract feature list from geojson
featureList = data['features']

#Use a for loop to update counties with state name and abbreviation
update_idx = 0
for f in featureList:
    stateName = fipsToState.get(f['properties']['STATE'])
    stateAbbrev = fipsToCode.get(f['properties']['STATE'])
    featureList[update_idx]['properties'].update({'CODE': stateAbbrev, 'STATENAME': stateName})
    update_idx += 1

# Overwrite geojson features with updated list
data['features'] = featureList

# Display properties in first element in features value to confirm state abbreviation and name is present
data['features'][0]['properties']    

{'GEO_ID': '0500000US02261',
 'STATE': '02',
 'COUNTY': '261',
 'NAME': 'Valdez-Cordova',
 'LSAD': 'CA',
 'CENSUSAREA': 34239.88,
 'CODE': 'AK',
 'STATENAME': 'Alaska'}

__Task 2:__ Find the _n_ most common names of counties.
- Display the nation-wide frequency of the _n_ most common county names
- List the county names and the states in which they can be found

In [2]:
# Create an empty dictionary to store county name and frequency
countyDict = {}

# Loop through each element in featureList, checking if the county name is in the dictionary.
# If present, increment frequency by one
# If not, add to dictionary with frequency = 1
for f in featureList:
    countyName = f['properties']['NAME'] 
    if countyDict.__contains__(countyName):
        curVal = countyDict.get(countyName)
        countyDict.update({countyName: curVal +1})
    else:
        countyDict.update({countyName: 1})
        
        
# Define function to display n most common names of counties
# In the event of a tie, print n+1 most common names of counties
def countyFrequency (countyDict, n):
    countyDict_sort = sorted(countyDict.items(), key = lambda item:item[1], reverse=True) # Sort countyDict by frequency
    # In the event of a tie for n most common counties
    if countyDict_sort[n-1][1] == countyDict_sort[n][1]:
        print('There is a tie for the {} most common counties in the US. The {} most common counties are displayed below:\n'.format(n, n+1))
        for i in range(n+1):
            print('There are {} {} counties in the US. They are found in the following states:'.format(countyDict_sort[i][1], countyDict_sort[i][0]))
            codeList = [None] * countyDict_sort[i][1]
            codeList_idx = 0 # Iterator for index
            for f in featureList:
                if f['properties']['NAME'] == countyDict_sort[i][0]:
                    codeList[codeList_idx] = (f['properties']['CODE']) # Add Abbreviation to list
                    codeList_idx += 1
                else:
                    pass
            print(codeList)
    # No tie
    else:
        for i in range(n):
            print('There are {} {} counties in the US. They are found in the following states:'.format(countyDict_sort[i][1], countyDict_sort[i][0]))
            codeList = [None] * countyDict_sort[i][1]
            codeList_idx = 0 # Iterator for index
            for f in featureList:
                if f['properties']['NAME'] == countyDict_sort[i][0]:
                    codeList[codeList_idx] = (f['properties']['CODE']) # Add Abbreviation to list
                    codeList_idx += 1
                else:
                    pass
            print(codeList)
        

countyFrequency output in the event of a tie (n=2):

In [3]:
countyFrequency(countyDict, 2)

There is a tie for the 2 most common counties in the US. The 3 most common counties are displayed below:

There are 31 Washington counties in the US. They are found in the following states:
['CO', 'FL', 'AR', 'IL', 'IA', 'ME', 'MD', 'MO', 'TN', 'OK', 'OR', 'WI', 'UT', 'VT', 'KS', 'IN', 'KY', 'MN', 'GA', 'AL', 'LA', 'ID', 'NY', 'NC', 'NE', 'MS', 'PA', 'OH', 'RI', 'VA', 'TX']
There are 26 Franklin counties in the US. They are found in the following states:
['GA', 'AL', 'IN', 'ID', 'KY', 'IA', 'MS', 'NY', 'TX', 'AR', 'FL', 'LA', 'ME', 'MA', 'IL', 'KS', 'MO', 'TN', 'NE', 'NC', 'PA', 'OH', 'VA', 'VA', 'WA', 'VT']
There are 26 Jefferson counties in the US. They are found in the following states:
['IL', 'GA', 'IA', 'OR', 'CO', 'AR', 'FL', 'KS', 'KY', 'AL', 'LA', 'MS', 'ID', 'IN', 'MO', 'TN', 'TX', 'NY', 'OK', 'NE', 'MT', 'PA', 'OH', 'WV', 'WI', 'WA']


countyFrequency output in the event of no tie (n=5):

In [4]:
countyFrequency(countyDict, 5)

There are 31 Washington counties in the US. They are found in the following states:
['CO', 'FL', 'AR', 'IL', 'IA', 'ME', 'MD', 'MO', 'TN', 'OK', 'OR', 'WI', 'UT', 'VT', 'KS', 'IN', 'KY', 'MN', 'GA', 'AL', 'LA', 'ID', 'NY', 'NC', 'NE', 'MS', 'PA', 'OH', 'RI', 'VA', 'TX']
There are 26 Franklin counties in the US. They are found in the following states:
['GA', 'AL', 'IN', 'ID', 'KY', 'IA', 'MS', 'NY', 'TX', 'AR', 'FL', 'LA', 'ME', 'MA', 'IL', 'KS', 'MO', 'TN', 'NE', 'NC', 'PA', 'OH', 'VA', 'VA', 'WA', 'VT']
There are 26 Jefferson counties in the US. They are found in the following states:
['IL', 'GA', 'IA', 'OR', 'CO', 'AR', 'FL', 'KS', 'KY', 'AL', 'LA', 'MS', 'ID', 'IN', 'MO', 'TN', 'TX', 'NY', 'OK', 'NE', 'MT', 'PA', 'OH', 'WV', 'WI', 'WA']
There are 24 Jackson counties in the US. They are found in the following states:
['AR', 'MO', 'WI', 'CO', 'FL', 'KS', 'MI', 'KY', 'MN', 'GA', 'LA', 'AL', 'MS', 'IL', 'IA', 'IN', 'TN', 'TX', 'OK', 'OR', 'NC', 'OH', 'SD', 'WV']
There are 24 Lincoln cou

__Task 3:__ Basic statistics by state
For each state, find:
- The number of counties
- The name and size (census area) of the biggest and smallest county by area
- The total and average area of counties

In [5]:
# Get a list of state codes using set comprehension. They are not consecutive and not (1, 51)
stateCodeList = {f['properties']['STATE'] for f in featureList}
        
# For each state code, get number of counties, smallest and largest county,
# and total county size
for s in stateCodeList:
    stateCounties = 0 # Initialize county list
    largestCounty = ['initialName', 0] # Initialize for largest county
    areaCounty_total = 0 # Initialize total county area
    areaCounty_avg = 0 # Initialize average county area
    stateName = fipsToState.get(s) # Get state name from FIPS to state dictionary
    stateAbbrev = fipsToCode.get(s) # Get state abbreviation from FIPS to code dictionary
    for f in featureList: # Initialize smallest county variable using the first county of each state 
        if f['properties']['STATE'] == s:
            smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            break # This populates the list with only one county, area pair
        else:
            pass
    for f in featureList: # Loop through feature list
        if f['properties']['STATE'] == s: # Filter to currently iterated state code
            stateCounties += 1 # County iterator for total county
            areaCounty_total += f['properties']['CENSUSAREA'] # Add county area to total
            if f['properties']['CENSUSAREA'] > largestCounty[1]: # Largest county comparison
                largestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            if f['properties']['CENSUSAREA'] < smallestCounty[1]: # Smallest county comparison. **Don't use elif**
                smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            else:
                pass
        else:
            pass
    areaCounty_avg = areaCounty_total / stateCounties # Average county area
        
    print('{} ({}) has {} counties.'.format(stateName, stateAbbrev, stateCounties))
    print('Its largest county is {} which is {:.0f} square miles.'.format(largestCounty[0], largestCounty[1])) # {:.0f} removes the decimal from a floating point number
    print('Its smallest county is {} which is {:.0f} square miles.'.format(smallestCounty[0], smallestCounty[1]))
    print('Its total county area is {:.0f} square miles.'.format(areaCounty_total))
    print('Its average county area is {:.0f} square miles.\n'.format(areaCounty_avg))

Delaware (DE) has 3 counties.
Its largest county is Sussex which is 936 square miles.
Its smallest county is New Castle which is 426 square miles.
Its total county area is 1949 square miles.
Its average county area is 650 square miles.

Illinois (IL) has 102 counties.
Its largest county is McLean which is 1183 square miles.
Its smallest county is Putnam which is 160 square miles.
Its total county area is 55519 square miles.
Its average county area is 544 square miles.

Pennsylvania (PA) has 67 counties.
Its largest county is Lycoming which is 1229 square miles.
Its smallest county is Montour which is 130 square miles.
Its total county area is 44743 square miles.
Its average county area is 668 square miles.

Louisiana (LA) has 64 counties.
Its largest county is Vernon which is 1328 square miles.
Its smallest county is Orleans which is 169 square miles.
Its total county area is 43204 square miles.
Its average county area is 675 square miles.

Michigan (MI) has 83 counties.
Its largest co

__Task 4:__ Save new geojson file with state abbreviations and full state names:

In [6]:
# Write new json file
with open("countiesUpdate.json", "w") as outfile:
    json.dump(data, outfile)